In [1]:
#%%
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact, FloatSlider, IntSlider, Layout, VBox

def calculate_costs(market_rent, percent_social, cost_inflation_rate, rent_inflation_rate, occupancy_social, occupancy_mixed,
                    initial_land_cost, initial_building_cost_per_unit, total_social_units, operating_cost_per_unit, social_rent, years=100):
    # Calculate the total building cost based on the number of social units
    initial_building_cost = initial_building_cost_per_unit * total_social_units
    
    # Recalculate the number of social units and buildings needed for mixed model
    social_units_mixed_per_building = total_social_units * percent_social
    buildings_needed_mixed = total_social_units / social_units_mixed_per_building
    total_units_mixed = buildings_needed_mixed * total_social_units

    # Calculate the number of market units in the mixed model
    market_units_mixed = total_units_mixed - total_social_units

    # Initial capital costs
    initial_capital_cost_pure = initial_land_cost + initial_building_cost
    initial_capital_cost_mixed = (initial_land_cost + initial_building_cost) * buildings_needed_mixed
    
    # Arrays to store costs over time
    pure_social_costs = []
    mixed_market_costs = []
    
    # Total initial costs
    pure_social_total_cost = initial_capital_cost_pure
    mixed_market_total_cost = initial_capital_cost_mixed

    for year in range(years):
        # Apply cost inflation to operating costs
        inflated_operating_cost_per_unit = operating_cost_per_unit * ((1 + cost_inflation_rate) ** year)
        # Annual operating costs
        annual_operating_cost_pure = inflated_operating_cost_per_unit * total_social_units
        annual_operating_cost_mixed = inflated_operating_cost_per_unit * total_units_mixed

        # Apply rent inflation to rents and calculate annual rent income, considering occupancy rates
        inflated_market_rent = market_rent * ((1 + rent_inflation_rate) ** year)
        inflated_social_rent = social_rent * ((1 + rent_inflation_rate) ** year)
        
        market_rent_income = inflated_market_rent * 12 * market_units_mixed * occupancy_mixed
        social_rent_income_pure = inflated_social_rent * 12 * total_social_units * occupancy_social
        social_rent_income_mixed = inflated_social_rent * 12 * total_social_units * occupancy_social
        
        annual_rent_income_pure = social_rent_income_pure
        annual_rent_income_mixed = market_rent_income + social_rent_income_mixed
        
        # Update total costs
        pure_social_total_cost += annual_operating_cost_pure - annual_rent_income_pure
        mixed_market_total_cost += annual_operating_cost_mixed - annual_rent_income_mixed

        # Append costs to lists
        pure_social_costs.append(pure_social_total_cost)
        mixed_market_costs.append(mixed_market_total_cost)

    return pure_social_costs, mixed_market_costs, market_units_mixed, initial_capital_cost_mixed

def find_breakeven_points(pure_social_costs, mixed_market_costs):
    # Find breakeven points where the cost lines intersect
    breakeven_points = []
    for year in range(1, len(pure_social_costs)):
        if (pure_social_costs[year - 1] > mixed_market_costs[year - 1] and pure_social_costs[year] < mixed_market_costs[year]) or \
           (pure_social_costs[year - 1] < mixed_market_costs[year - 1] and pure_social_costs[year] > mixed_market_costs[year]):
            breakeven_points.append(year)
    return breakeven_points

def find_construction_cost_reach_point(pure_social_costs, initial_capital_cost_mixed):
    # Find the year when Pure Social Housing costs reach the construction costs of Mixed Market Housing
    for year, cost in enumerate(pure_social_costs):
        if cost >= initial_capital_cost_mixed:
            return year
    return None

def interactive_graph(market_rent, percent_social, cost_inflation_rate, rent_inflation_rate, occupancy_social, occupancy_mixed,
                      initial_land_cost, initial_building_cost_per_unit, total_social_units, operating_cost_per_unit, social_rent):
    pure_social_costs, mixed_market_costs, market_units_mixed, initial_capital_cost_mixed = calculate_costs(
        market_rent, percent_social, cost_inflation_rate, rent_inflation_rate, occupancy_social, occupancy_mixed,
        initial_land_cost, initial_building_cost_per_unit, total_social_units, operating_cost_per_unit, social_rent, years=100
    )
    
    # Find breakeven points
    breakeven_points = find_breakeven_points(pure_social_costs, mixed_market_costs)
    
    # Find when Pure Social Housing costs reach the construction costs of the Mixed Market option
    construction_cost_reach_point = find_construction_cost_reach_point(pure_social_costs, initial_capital_cost_mixed)
    
    # Create the figure
    fig = go.Figure()

    # Pure Social Housing Costs
    fig.add_trace(go.Scatter(
        x=list(range(1, 101)), y=pure_social_costs, 
        mode='lines', 
        name='Pure Social Housing',
        line=dict(color='royalblue', width=4)
    ))

    # Mixed Market Housing Costs
    fig.add_trace(go.Scatter(
        x=list(range(1, 101)), y=mixed_market_costs, 
        mode='lines', 
        name='Mixed Market Housing',
        line=dict(color='firebrick', width=4, dash='dash')
    ))

    # Add initial build price line for mixed market
    fig.add_trace(go.Scatter(
        x=list(range(1, 101)), 
        y=[initial_capital_cost_mixed] * 100, 
        mode='lines', 
        name='Initial Mixed Market Build Cost',
        line=dict(color='green', width=2, dash='dot')
    ))

    # Add breakeven points
    for point in breakeven_points:
        fig.add_trace(go.Scatter(
            x=[point], y=[pure_social_costs[point]],
            mode='markers+text',
            name='Breakeven Point',
            text=[f'Year {point}'],
            textposition='top center',
            marker=dict(color='green', size=10, symbol='x')
        ))

    # Add construction cost reach point
    if construction_cost_reach_point is not None:
        fig.add_trace(go.Scatter(
            x=[construction_cost_reach_point], y=[initial_capital_cost_mixed],
            mode='markers+text',
            name='Mixed Market Build-Equivalent Point',
            text=[f'Year {construction_cost_reach_point}'],
            textposition='top center',
            marker=dict(color='purple', size=10, symbol='circle')
        ))

    # Layout for the graph
    fig.update_layout(
        title={
            'text': f'Cost Comparison of 69 Social Housing Units<br><sup>Market Units in Mixed: {int(market_units_mixed)}</sup>',
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        xaxis_title='Years',
        yaxis_title='Total Cost ($)',
        legend_title='Housing Type',
        template='plotly_white',
        font=dict(
            family="Arial, sans-serif",
            size=14,
            color="black"
        ),
        title_font_size=20,
        margin=dict(l=40, r=40, t=80, b=40)
    )
    
    fig.show()

# Adjust slider layout for a more compact and visually consistent design
slider_layout = Layout(width='450px', description_width='150px')

# Create interactive controls with appealing styles
controls = VBox([
    FloatSlider(value=1104.00, min=500.0, max=2000.0, step=50.0, description='Market Rent ($)', layout=slider_layout, style={'description_width': 'initial'}),
    FloatSlider(value=0.25, min=0.1, max=0.9, step=0.05, description='Social % in Mixed', layout=slider_layout, style={'description_width': 'initial'}),
    FloatSlider(value=0.02, min=0.00, max=0.1, step=0.005, description='Cost Inflation Rate (%)', layout=slider_layout, style={'description_width': 'initial'}),
    FloatSlider(value=0.02, min=0.00, max=0.1, step=0.005, description='Rent Inflation Rate (%)', layout=slider_layout, style={'description_width': 'initial'}),
    FloatSlider(value=1.0, min=0.5, max=1.0, step=0.05, description='Occupancy Rate Social (%)', layout=slider_layout, style={'description_width': 'initial'}),
    FloatSlider(value=1.0, min=0.5, max=1.0, step=0.05, description='Occupancy Rate Market (%)', layout=slider_layout, style={'description_width': 'initial'}),
    FloatSlider(value=3000000, min=1000000, max=10000000, step=100000, description='Initial Land Cost ($)', layout=slider_layout, style={'description_width': 'initial'}),
    FloatSlider(value=11348025 / 69, min=50000, max=500000, step=5000, description='Building Cost per Unit ($)', layout=slider_layout, style={'description_width': 'initial'}),
    IntSlider(value=69, min=10, max=200, step=1, description='Total Social Units', layout=slider_layout, style={'description_width': 'initial'}),
    FloatSlider(value=9814.71, min=5000, max=20000, step=500, description='Operating Cost per Unit ($)', layout=slider_layout, style={'description_width': 'initial'}),
    FloatSlider(value=292.41, min=100, max=1000, step=50, description='Social Rent ($)', layout=slider_layout, style={'description_width': 'initial'}),
])

# Interactive display
interact(interactive_graph, 
         market_rent=controls.children[0],
         percent_social=controls.children[1],
         cost_inflation_rate=controls.children[2],
         rent_inflation_rate=controls.children[3],
         occupancy_social=controls.children[4],
         occupancy_mixed=controls.children[5],
         initial_land_cost=controls.children[6],
         initial_building_cost_per_unit=controls.children[7],
         total_social_units=controls.children[8],
         operating_cost_per_unit=controls.children[9],
         social_rent=controls.children[10])

# %%



interactive(children=(FloatSlider(value=1104.0, description='Market Rent ($)', layout=Layout(width='450px'), m…

<function __main__.interactive_graph(market_rent, percent_social, cost_inflation_rate, rent_inflation_rate, occupancy_social, occupancy_mixed, initial_land_cost, initial_building_cost_per_unit, total_social_units, operating_cost_per_unit, social_rent)>